In [1]:
import os
import sys
import math
import locale
import pypdf
sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np

from typing import List
from pydantic import BaseModel, Field
from utils_ccd import get_connection, get_info_file_path, extract_text_from_pdf, generate_pdf_office
from datetime import datetime
from dotenv import load_dotenv
from docxtpl import DocxTemplate

load_dotenv()
conn = get_connection()
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from antecedentes import create_antecedentes_doc
from pathlib import Path
from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate
from docx2pdf import convert

llm = AzureChatOpenAI(model_name="gpt-4o")

c:\Users\05911205424\Documents\Dev\ccd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
antecedentes_sql = ''' 
SELECT DISTINCT pro.numero_processo,
       pro.ano_processo,
       pro.assunto,
       pro.interessado,
       pm.Descricao as marcador      
       
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = edp.IDPessoa
WHERE pro.setor_atual = 'CCD'
AND lower(pm.Descricao) like '%antecedente%'
'''
processos_antecedentes = pd.read_sql(antecedentes_sql, conn)

In [4]:
sql_info_antecedentes_gaana = ''' 
WITH RankedData AS (
    SELECT 
    	ai.setor,
        concat(pro.numero_processo, '/', pro.ano_processo) as processo,
        pro.assunto,
        pro.interessado,
        concat(rtrim(ai.setor),'_',
        ai.numero_processo ,'_',
        ai.ano_processo,'_',RIGHT(concat('0000',ai.ordem),4),'.pdf') as arquivo,
        ai.datapublicacao, 
        ppe.SequencialProcessoEvento, 
        ai.numero_processo, 
        ai.ano_processo,
        ROW_NUMBER() OVER (
            PARTITION BY CONCAT(ai.numero_processo, '/', ai.ano_processo)
            ORDER BY ppe.SequencialProcessoEvento DESC
        ) AS rn
    FROM processo.dbo.vw_Ata_Informacao ai 
    LEFT JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON ai.idInformacao = ppe.IdInformacao
    LEFT JOIN processo.dbo.Processos pro
    	ON pro.numero_processo = ai.numero_processo AND pro.ano_processo = ai.ano_processo
    WHERE CONCAT(ai.numero_processo, '/', ai.ano_processo) IN (
        SELECT CONCAT(pro.numero_processo, '/', pro.ano_processo)
        FROM processo.dbo.Processos pro
        INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
            ON pmp.IdProcesso = pro.IdProcesso
        INNER JOIN processo.dbo.Pro_Marcador pm 
            ON pmp.IdMarcador = pm.IdMarcador
        LEFT JOIN processo.dbo.Exe_Debito ed 
            ON ed.IdProcessoExecucao = pro.IdProcesso
        LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
            ON edp.IDDebito = ed.IdDebito
        LEFT JOIN processo.dbo.GenPessoa gp 
            ON gp.IdPessoa = edp.IDPessoa
        WHERE pro.setor_atual = 'CCD'
        AND lower(pm.Descricao) LIKE '%antecedente%'
    )
    AND ai.setor LIKE '%GAANA%'
)
SELECT * 
FROM RankedData 
WHERE rn = 1
ORDER BY numero_processo, ano_processo;

'''
df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)

In [28]:
def get_transito(nome):
    with open("../consultas/processos_transito_cpf.sql") as f:
        query = f.read()
    q = query.format(nome=nome)
    df = pd.read_sql(q, get_connection())
    df['valor_original'] = df['valor_original'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
    df['valor_atualizado'] = df['valor_atualizado'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
    df.fillna('', inplace=True)
    return df

def get_cpf_responsavel(nome_pessoa, numero_processo, ano_processo):
    query = f'''
SELECT gp.Documento as cpf
	FROM Processo.dbo.Processos p
		INNER JOIN Processo.dbo.Pro_ProcessosResponsavelDespesa pprd 
		ON pprd.IdProcesso = p.IdProcesso
		INNER JOIN Processo.dbo.GenPessoa gp 
		ON gp.IdPessoa = pprd.IdPessoa
	WHERE p.numero_processo = '{numero_processo}'
	AND p.ano_processo = '{ano_processo}'
	AND lower(gp.Nome) = '{nome_pessoa.lower()}'
 '''
    df = pd.read_sql(query, get_connection())

    if len(df) > 1:
        # Corrigido: cpf não existia nesse escopo
        raise ValueError(f"Mais de um CPF para {nome_pessoa}: {df['cpf'].tolist()}")
    elif len(df) == 0:
        return ''
    return df['cpf'].values[0] or ''
    
def get_valores_pessoa(row):
    pessoa_strc = row['pessoas']
    print(f'Pessoas: {pessoa_strc.pessoas}')
    valores = []
    for pessoa in pessoa_strc.pessoas:
        nome_pessoa = pessoa.nome_pessoa
        transitos = get_transito(nome_pessoa)
        cpfs = transitos['cpf'].unique()
        if len(cpfs) > 1:
            print(f'Mais de um cpf por pessoa')
            cpf = ''
        elif len(cpfs) == 0:
            cpf = get_cpf_responsavel(nome_pessoa, row['numero_processo'], row['ano_processo'])
        else:
            cpf = cpfs[0]
        valores.append({'nome': nome_pessoa, 'cpf': cpf, 'transitos': transitos.to_dict(orient='records')})
    return valores

def valores_vazios(valores):
    return all([not bool(v['transitos']) for v in valores])


class PessoaAntecedentes(BaseModel):
  """Pessoa cujo antecedente deve ser buscado"""
  nome_pessoa: str = Field(description="Nome da pessoa para buscar antecedentes")

class PessoasAntecedentes(BaseModel):
  """Lista de pessoas cujos antecedentes devem ser buscados"""
  pessoas: List[PessoaAntecedentes] = Field(description="Lista de pessoas para buscar antecedentes")
  
def get_pessoas_llm(texto_despacho):

  prompt = PromptTemplate.from_template("""
    Você é um agente que identifica listas de pessoas em requisições de antecedentes. 
      Você recebeu uma requisição para buscar antecedentes de pessoas em um processo. Remova qualquer vocativo do nome da pessoa, tal como Sr., Sra., Dr., Dra., etc.
      Não inclua nomes de conselheiros, advogados ou partes do processo. Ex: Sra. Conselheira Substituta Ana
Paula de Oliveira Gomes
      O texto da requisição é o seguinte:
      "{input}"
      Encontre os antecedentes das pessoas listadas na requisição.

    Sua resposta:
    """)

  structured_llm = llm.with_structured_output(schema=PessoasAntecedentes)
  chain = prompt | structured_llm
  return chain.invoke(texto_despacho)
    

In [6]:
df_info_gaana['caminho_arquivo'] = df_info_gaana.apply(get_info_file_path, axis=1)
df_info_gaana['texto_pdf'] = df_info_gaana['caminho_arquivo'].apply(extract_text_from_pdf)
df_info_gaana['pessoas'] = df_info_gaana['texto_pdf'].apply(get_pessoas_llm)

In [29]:
df_info_gaana['valores'] = df_info_gaana.apply(get_valores_pessoa, axis=1)

Pessoas: [PessoaAntecedentes(nome_pessoa='Norma Ferreira Caldas')]
Pessoas: [PessoaAntecedentes(nome_pessoa='Atevaldo Nazário da Silva')]
Pessoas: [PessoaAntecedentes(nome_pessoa='Francisco das Chagas Félix')]
Pessoas: [PessoaAntecedentes(nome_pessoa='MARCOS ANTONIO DE OLIVEIRA')]
Mais de um cpf por pessoa
Pessoas: [PessoaAntecedentes(nome_pessoa='Dagoberto Bessa Cavalcante')]


In [31]:
df_info_gaana

,setor,processo,assunto,interessado,arquivo,datapublicacao,SequencialProcessoEvento,numero_processo,ano_processo,rn,caminho_arquivo,texto_pdf,pessoas,valores
0,GAANA,004382/2024,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",PREFEITURA MUNICIPAL DE SÃO JOSÉ DE MIPIBÚ ...,GAANA_004382_2024_0021.pdf,2025-09-30 07:43:59.980,40,004382,2024,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 4382/...,pessoas=[PessoaAntecedentes(nome_pessoa='Norma...,"[{'nome': 'Norma Ferreira Caldas', 'cpf': '971..."
1,GAANA,004452/2023,CONTAS DO CHEFE DO PODER EXECUTIVO DA PREFEITU...,PREFEITURA MUNICIPAL DE ENCANTO ...,GAANA_004452_2023_0014.pdf,2025-09-30 11:54:26.253,27,004452,2023,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 4452/...,pessoas=[PessoaAntecedentes(nome_pessoa='Ateva...,"[{'nome': 'Atevaldo Nazário da Silva', 'cpf': ..."
2,GAANA,005118/2021,CONTAS ANUAIS DE GESTÃO - OMISSÃO - EXERCÍCIO ...,PREFEITURA MUNICIPAL DE AFONSO BEZERRA ...,GAANA_005118_2021_0048.pdf,2025-09-30 12:48:42.947,88,005118,2021,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 5118/...,pessoas=[PessoaAntecedentes(nome_pessoa='Franc...,"[{'nome': 'Francisco das Chagas Félix', 'cpf':..."
3,GAANA,010115/2016,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.PARAZINHO ...,GAANA_010115_2016_0028.pdf,2025-09-29 12:18:11.417,49,010115,2016,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 10115...,pessoas=[PessoaAntecedentes(nome_pessoa='MARCO...,"[{'nome': 'MARCOS ANTONIO DE OLIVEIRA', 'cpf':..."
4,GAANA,010145/2016,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.SEVERIANO MELO ...,GAANA_010145_2016_0026.pdf,2025-10-02 08:41:09.223,50,010145,2016,1,\\10.24.0.6\tce$\Informacoes_PDF\GAANA\GAANA_0...,\n \n1 \n \n \nProcesso nº: 10145...,pessoas=[PessoaAntecedentes(nome_pessoa='Dagob...,"[{'nome': 'Dagoberto Bessa Cavalcante', 'cpf':..."


In [32]:
#df_info_gaana['len_valores'] = df_info_gaana['valores'].apply(lambda x: len(x))
df_info_gaana['sem_valores'] = df_info_gaana['valores'].apply(valores_vazios)
antecedentes = df_info_gaana

In [35]:
for row in antecedentes.itertuples():
    doc = DocxTemplate("templates/antecedentes.docx")
    #valores = [v for v in row.valores if len(v['transitos']) > 0]
    nomes = [v['nome'] for v in row.valores]
    if len(nomes) > 1:
        responsaveis = ', '.join(nomes[:-1]) + ' e ' + nomes[-1]
    else:
        responsaveis = nomes[0] if nomes else ''
    for v in row.valores:
        if v['cpf'] is None:
            v['cpf'] = ''
    context = {
        'processo': row.processo,
        'assunto': row.assunto,
        'interessado': row.interessado,
        'valores': row.valores,
        'responsaveis': responsaveis,
        'data': datetime.now().strftime("%d/%m/%Y"),
        'sem_valores': row.sem_valores
    }
    processo_underline = row.processo.replace('/', '_')

    doc.render(context)
    doc.save(f"saidas/antecedentes/{processo_underline}.docx")
    generate_pdf_office(f"saidas/antecedentes/{processo_underline}.docx", f"saidas/antecedentes/")
    print(f"Gerado arquivo para o processo {row.processo}")

Gerado arquivo para o processo 004382/2024
Gerado arquivo para o processo 004452/2023
Gerado arquivo para o processo 005118/2021
Gerado arquivo para o processo 010115/2016
Gerado arquivo para o processo 010145/2016
